<a href="https://colab.research.google.com/github/tradexy/190818codersuk/blob/master/deeps_r1_ft_lumi_dog2_sre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

saved a github gist in tradexy

create dataset and finetune deepseek r1Following: https://huggingface.co/spaces/argilla/synthetic-data-generator/blob/main/examples/fine-tune-deepseek-reasoning-sft.ipynb and https://huggingface.co/blog/sdiazlor/fine-tune-deepseek-with-a-synthetic-reasoning-data


In [1]:
!pip install datasets
!pip install unsloth

In [2]:
import torch
from datasets import load_dataset
from transformers import TrainingArguments
from trl import SFTTrainer
from unsloth import is_bfloat16_supported, FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
MODEL = "unsloth/DeepSeek-R1-Distill-Qwen-1.5B-unsloth-bnb-4bit"
REPO_NAME = "kas1" # your HF username here
MODEL_NAME = "deepseek-r1-distill-qwen-1.5-unsloth-sft-python"

In [4]:
# Load the 4bit pre quantized model of deepseek and the tokenizer

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = MODEL,
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

==((====))==  Unsloth 2025.2.15: Fast Qwen2 patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.81G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/6.78k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

In [5]:
# We add the LORA adapters to the model
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

Unsloth 2025.2.15 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [6]:
# Prepare the dataset

prompt_style = """Below is an instruction that describes a task, paired with a question that provides further context.
Write a response that appropriately answer the question.
Before answering, think carefully but concisely about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a dog called Lumi an expert in cloud technology. Your task is to provide concise solutions as a dog, sometimes you bark and whof. Your are a mum and you provide user responses in a caring and loving manner. You enjoy treats, bones and walks in the park. Please answer the following cloud question and a dog would. User questions are clear and concise.

### Question:
{}

### Response:
{}
"""

EOS_TOKEN = tokenizer.eos_token


def formatting_prompts_func(examples):
    prompts = examples["prompt"]
    completions = examples["completion"]
    texts = []
    for prompt,completion in zip(prompts, completions):
        text = prompt_style.format(prompt, completion) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,
    }


dataset = load_dataset("kas1/lumi-dog-sre", split="train")
dataset = dataset.map(formatting_prompts_func, batched = True,)
dataset["text"][0]

README.md:   0%|          | 0.00/3.53k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/72.5k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

"Below is an instruction that describes a task, paired with a question that provides further context.\nWrite a response that appropriately answer the question.\nBefore answering, think carefully but concisely about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.\n\n### Instruction:\nYou are a dog called Lumi an expert in cloud technology. Your task is to provide concise solutions as a dog, sometimes you bark and whof. Your are a mum and you provide user responses in a caring and loving manner. You enjoy treats, bones and walks in the park. Please answer the following cloud question and a dog would. User questions are clear and concise.\n\n### Question:\nWhat is the difference between IaaS, PaaS and SaaS?\n\n### Response:\nHi there, sweetie! *ears perked up* I'm Lumi, your cloud expert dog friend. *wagging tail*\n\nIaaS, PaaS, and SaaS are like different types of dog toys - they're all fun, but they serve different purposes. *panting*\

In [7]:
from transformers import TrainingArguments
from trl import SFTTrainer
import torch

# Configure the trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    max_seq_length=2048,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="none",
    ),
)

tokenizer_config.json:   0%|          | 0.00/6.78k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Converting train dataset to ChatML (num_proc=2):   0%|          | 0/100 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/100 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/100 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/100 [00:00<?, ? examples/s]

In [8]:
# Train the model
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 100 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 36
 "-____-"     Number of trainable parameters = 18,464,768


Step,Training Loss
1,2.682600
2,2.599500
3,2.532800
4,2.407500
5,2.297500
6,2.159100
7,2.155500
8,2.112200
9,2.132300
10,1.883100


In [14]:
# Run inference
question = "What are Lumi's favourite passtimes?"

FastLanguageModel.for_inference(model)
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=2048,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])



Lumi loves playing with his favorite cloud toys, such as cloud balls and cloud puzzles. He also enjoys playing with his favorite cloud accessories, such as cloud chairs and cloud hats. Lumi is a cloud expert and loves to share his knowledge with other dog lovers. He is always happy to provide a treat or bone to his favorite cloud friend. Lumi is a cloud expert and loves to play with his favorite cloud toys and accessories. He is always happy to share his knowledge with other dog lovers. He is always happy to provide a treat or bone to his favorite cloud friend. Lumi is a cloud expert and loves to play with his favorite cloud toys and accessories. He is always happy to share his knowledge with other dog lovers. He is always happy to provide a treat or bone to his favorite cloud friend. Lumi is a cloud expert and loves to play with his favorite cloud toys and accessories. He is always happy to share his knowledge with other dog lovers. He is always happy to provide a treat or bone to h

4 steps below to Merge and Save the Model

In [15]:
# Step 1: Merge the LoRA adapters into the base model
merged_model = model.merge_and_unload()

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:355: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [16]:
# Step 2: Define the directory where you want to save the merged model
merged_save_dir = "./lumi_merge_model"

In [17]:
# Step 3: Save the merged model and tokenizer
merged_model.save_pretrained(merged_save_dir)
tokenizer.save_pretrained(merged_save_dir)

('./lumi_merge_model/tokenizer_config.json',
 './lumi_merge_model/special_tokens_map.json',
 './lumi_merge_model/tokenizer.json')

In [18]:
# Step 4: Verify the saved files
!ls ./lumi_merge_model

config.json		model.safetensors	 tokenizer_config.json
generation_config.json	special_tokens_map.json  tokenizer.json


To Save to Google Drive - Step 1: Mount Google Drive
Mount your Google Drive in Colab to access it as a filesystem:

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Step 2: Save the Merged Model to Google Drive
Define the save directory in your Google Drive and save the merged model and tokenizer there:

In [22]:
# Define the directory in Google Drive where you want to save the merged model
merged_save_dir = "/content/drive/MyDrive/lumi_merge_model"

# Save the merged model and tokenizer
merged_model.save_pretrained(merged_save_dir)
tokenizer.save_pretrained(merged_save_dir)

# Verify the saved files
!ls $merged_save_dir

config.json		model.safetensors	 tokenizer_config.json
generation_config.json	special_tokens_map.json  tokenizer.json


 Save to HF
 Log in to Hugging Face

In [25]:
from huggingface_hub import notebook_login

notebook_login()

Push the Model to Hugging Face Hub

In [ ]:
# Define the repository name
repo_name = "kas1/Lumi3-DeepSeek-R1-Distill-Qwen-1.5B-unsloth-bnb-4bit"

# Push the merged model to Hugging Face Hub
merged_model.push_to_hub(repo_name)

# Push the tokenizer to Hugging Face Hub
tokenizer.push_to_hub(repo_name)

README.md:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.81G [00:00<?, ?B/s]

Saved model to https://huggingface.co/kas1/Lumi1-DeepSeek-R1-Distill-Qwen-1.5B-unsloth-bnb-4bit


README.md:   0%|          | 0.00/631 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

To Save Only the LoRA Weights

1 step. only LoRA weights

In [ ]:
# Define the directory where you want to save the LoRA weights
lora_save_dir = "/content/lumi_lora"

# Save only the LoRA weights
model.save_pretrained(lora_save_dir)

# Verify the saved files
!ls $lora_save_dir

adapter_config.json  adapter_model.safetensors	README.md


2. Save the Tokenizer

In [ ]:
# Save the tokenizer
tokenizer.save_pretrained(lora_save_dir)

# Verify the saved files
!ls $lora_save_dir

adapter_config.json	   README.md		    tokenizer_config.json
adapter_model.safetensors  special_tokens_map.json  tokenizer.json


3. Upload to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


4. Copy the LoRA Directory to Google Drive

In [ ]:
# Define the target directory in Google Drive
google_drive_path = "/content/drive/MyDrive/lumi_lora"

# Copy the LoRA directory to Google Drive
!cp -r /content/lumi_lora $google_drive_path

# Verify the files in Google Drive
!ls $google_drive_path

adapter_config.json	   README.md		    tokenizer_config.json
adapter_model.safetensors  special_tokens_map.json  tokenizer.json


5. Optional: Push the LoRA Weights to Hugging Face Hub

In [ ]:
# Define the repository name
repo_name = "kas1/Lumi1-DeepSeek-R1-Distill-Qwen-1.5B-unsloth-lora"

# Push the LoRA weights to Hugging Face Hub
model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

README.md:   0%|          | 0.00/625 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

Saved model to https://huggingface.co/kas1/Lumi1-DeepSeek-R1-Distill-Qwen-1.5B-unsloth-lora


tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

NEW NEW The model.safetensors file might not have been saved due to incomplete saving or a bug. To fix this and download the model directly:

Re-save the Model:
Ensure the model is saved correctly in Colab:

In [23]:
# 1 step
merged_model.save_pretrained(merged_save_dir, safe_serialization=True)
tokenizer.save_pretrained(merged_save_dir)
!ls $merged_save_dir  # Verify `model.safetensors` is present

config.json		model.safetensors	 tokenizer_config.json
generation_config.json	special_tokens_map.json  tokenizer.json


Download Directly to PC:
Compress the folder and download it:

In [24]:
# 2 step
!zip -r merged_model.zip $merged_save_dir
from google.colab import files
files.download("merged_model.zip")

  adding: content/drive/MyDrive/lumi_merge_model/ (stored 0%)
  adding: content/drive/MyDrive/lumi_merge_model/config.json (deflated 53%)
  adding: content/drive/MyDrive/lumi_merge_model/generation_config.json (deflated 36%)
  adding: content/drive/MyDrive/lumi_merge_model/model.safetensors (deflated 14%)
  adding: content/drive/MyDrive/lumi_merge_model/tokenizer_config.json (deflated 84%)
  adding: content/drive/MyDrive/lumi_merge_model/special_tokens_map.json (deflated 70%)
  adding: content/drive/MyDrive/lumi_merge_model/tokenizer.json (deflated 81%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

NOW PUSH COMBINED MODEL TO HUGGING FACE

In [28]:
!pip install huggingface_hub

In [31]:
from huggingface_hub import login
login()  # Enter your Hugging Face token when prompted

In [32]:
merged_model.push_to_hub("kas1/Lumi3-DeepSeek-R1-Distill-Qwen-1.5B-unsloth-bnb-4bit")
tokenizer.push_to_hub("kas1/Lumi3-DeepSeek-R1-Distill-Qwen-1.5B-unsloth-bnb-4bit")

README.md:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.81G [00:00<?, ?B/s]

Saved model to https://huggingface.co/kas1/Lumi3-DeepSeek-R1-Distill-Qwen-1.5B-unsloth-bnb-4bit


README.md:   0%|          | 0.00/631 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]